In [1]:
import replicate
import os
from dotenv import load_dotenv
load_dotenv()  # This loads the environment variables from .env

my_prompt= """
    full_text: Control Learning for Soft Robotics
    Introduction 

    Classically controlled robots have revolutionised assembly lines where the environment is restricted and predictable. However, this control scheme has proven less effective in uncertain, unstructured environments due to the unmodelled nonlinearities in the morphology and its interaction with the environment. Embodied intelligence suggests that we are not controlled centrally but rather that the morphology and environment also contribute to behaviour. Therefore, the passive dynamics of the robot could be exploited to simplify the controller by making the passive dynamics closer to the desired behaviour. Consequently, there has been a growing interest in soft robots which have many passive degrees of freedom and are often underactuated. However, classical control necessitates exact kinematic and dynamic models which are hard to derive analytically for soft robots due to the nonlinear dynamics of the body, external forces and uncertain, unstructured environment. Model-free approaches are a potential solution to these problems as they can approximate inverse kinematic and dynamic models that accounts for the unknown nonlinearities.

    Control Learning for Locomotion

    Control for locomotion of soft robots can also be learned. Vibration-driven tensegrity robots present a challenge as the motion is unpredictable and therefore have no known analytical solution for generating gait.
    X used an evolutionary algorithm to optimise the motor frequencies for the control of a physical vibration-driven tensegrity robot and exploited the inherent resonance of the tensegrity structures to increase the locomotion speed. Y also aimed to exploit the resonance of a vibration-driven tensegrity robot to increase locomotion speed. However, they used Bayesian optimisation to efficiently learn control for a physical robot. The policy is learned by optimising three pulse width modulation values that control the input voltage of the vibrating motors. Bayesian optimisation models the objective function using regression and uses this to select the next point to acquire. Comparing the results between random search and Bayesian optimisation with and without a prior, they found the Bayesian optimisation with a prior produced the fastest speed using the same number of trials.

    input_text: X used an evolutionary algorithm to optimise the motor frequencies for the control of a physical vibration-driven tensegrity robot and exploited the inherent resonance of the tensegrity structures to increase the locomotion speed.
"""

my_system_prompt = """
    Given the full_text and a specific input_text, perform the following steps:
    1. **Extract Key Topics**:
    - Identify the primary subjects or technologies discussed in the input text. Focus on broad but relevant terms that are likely to be used in academic literature.
    2. **Identify Contextual Keywords**:
    - Determine additional terms that provide context or are closely related to the main topics. These should include specific technologies, problems, or methodologies mentioned in the text.
    3. **Determine the Type of Information Desired**:
    - Analyze the text to understand what kind of information is being sought (e.g., methodologies, applications, theoretical frameworks, case studies).
    Using the information gathered above, output a Google Scholar search query using only the most relevant keywords and logical operators. The query should be formatted as follows:
    "[Key Topics] AND [Contextual Keywords] AND ([Type of Information Desired])"
    Ensure that the query is balanced between specificity and breadth to maximize the chance of retrieving relevant academic papers.
    IMPORTANT: Only output the query formatted as above, do not write any additional details.
"""

inputs ={
    "top_k": 0,
    "top_p": 0.99,
    "prompt": my_prompt,
    "max_tokens": 32,
    "temperature": 0.4,
    "system_prompt": my_system_prompt,
    "length_penalty": 1,
    "stop_sequences": "<|end_of_text|>,<|eot_id|>",
    "prompt_template": "<|begin_of_text|><|start_header_id|>system<|end_header_id|>\n\n{system_prompt}<|eot_id|><|start_header_id|>user<|end_header_id|>\n\n{prompt}<|eot_id|><|start_header_id|>assistant<|end_header_id|>\n\n",
    "presence_penalty": 0,
    "log_performance_metrics": False
}


In [2]:
outputs = []
for event in replicate.stream(
    "meta/meta-llama-3-8b-instruct",
    input=inputs,
):
    outputs.append(str(event))

full_text = ''.join(outputs)
cleaned_text = full_text.replace('"', '').replace("\\", "").strip()
cleaned_text

'Soft Robotics AND Vibration-Driven Tensegrity Robot AND Control Learning'

In [63]:
import requests
from pprint import pprint

In [92]:
def fetch_sorted_data(query, rows=100, rows_per_query=20, max_tries=10):
    url = "https://api.crossref.org/works"
    entries = []
    offset = 0
    tries = 0

    while len(entries) < rows and tries < max_tries:
        params = {
            'query': query,
            'filter': 'type:journal-article',
            'filter': 'has-title:true',
            'filter': 'has-abstract:true',
            'rows': rows_per_query,
            'offset': offset
        }
        response = requests.get("https://api.crossref.org/works", params=params)
        if response.status_code == 200:
            data = response.json()
            entries.extend(data['message']['items']ems)
            offset += rows_per_query  # Increase offset to fetch the next batch of results
        else:
            print(f"Failed to fetch data: {response.status_code}")
            break  # Exit loop if there's an HTTP error
        
        tries += 1

    return entries[:rows]  # Return only the first 10 entries

In [93]:
# Fetch top 100 relevant entries
data = fetch_sorted_data(cleaned_text)

# Sort by citation count and date
sorted_by_citations = sorted(data, key=lambda x: x.get('is-referenced-by-count', 0), reverse=True)
sorted_by_date = sorted(data, key=lambda x: x['issued'].get('date-parts', 0), reverse=True)

print("Top 10 by Relevance Count:")
for entry in data[:10]:
    print(entry['title'][0])

print("\nTop 10 by Citation Count:")
for entry in sorted_by_citations[:10]:
    print(entry['title'][0])

print("\nTop 10 by Date:")
for entry in sorted_by_date[:10]:
    print(entry['title'][0])

Top 10 by Relevance Count:
Tensegrity metamaterials for soft robotics
Soft Spherical Tensegrity Robot Design Using Rod-Centered Actuation and Control
Design, Modeling, and Visual Learning-Based Control of Soft Robotic Fish Driven by Super-Coiled Polymers
LMI Control Design With Input Covariance Constraint for a Tensegrity Simplex Structure
Development of a Compliant Gripper Driven by 3 DOF Soft Robot
Fluid-Driven Soft Actuators for Soft Robots
Kinematics, Dynamics and Control of a Planar 3-DOF Tensegrity Robot Manipulator
Optimal impedance control based on body inertia for a hydraulically driven hexapod robot walking on uneven and extremely soft terrain
Locomotion characteristics of a wheeled vibration-driven robot with an enhanced pantograph-type suspension
Inertia-Driven Controlled Passive Actuation

Top 10 by Citation Count:
Soft robotics: Technologies and systems pushing the boundaries of robot abilities
Soft Robotics: Review of Fluid‐Driven Intrinsically Soft Devices; Manufacturin

In [94]:
import requests
from pprint import pprint
def fetch_sorted_data(query, rows=100, rows_per_query=20, max_tries=10):
    url = "https://api.crossref.org/works"
    entries = []
    offset = 0
    tries = 0

    while len(entries) < rows and tries < max_tries:
        params = {
            'query': query,
            'filter': 'type:journal-article',
            'filter': 'has-title:true',
            'filter': 'has-abstract:true',
            'rows': rows_per_query,
            'offset': offset
        }
        response = requests.get("https://api.crossref.org/works", params=params)
        if response.status_code == 200:
            data = response.json()
            items = [item for item in data['message']['items']]
            entries.extend(data['message']['items'])
            offset += rows_per_query  # Increase offset to fetch the next batch of results
        else:
            print(f"Failed to fetch data: {response.status_code}")
            break  # Exit loop if there's an HTTP error
        
        tries += 1

    return entries[:rows]  # Return only the first 10 entries

# Fetch top 100 relevant entries
data = fetch_sorted_data(cleaned_text)

# Sort by citation count and date
sorted_by_citations = sorted(data, key=lambda x: x.get('is-referenced-by-count', 0), reverse=True)
sorted_by_date = sorted(data, key=lambda x: x['issued'].get('date-parts', 0), reverse=True)

print("Top 10 by Relevance Count:")
for entry in data[:10]:
    print(entry['title'][0])

print("\nTop 10 by Citation Count:")
for entry in sorted_by_citations[:10]:
    print(entry['title'][0])

print("\nTop 10 by Date:")
for entry in sorted_by_date[:10]:
    print(entry['title'][0])


Top 10 by Relevance Count:
Tensegrity metamaterials for soft robotics
Soft Spherical Tensegrity Robot Design Using Rod-Centered Actuation and Control
Design, Modeling, and Visual Learning-Based Control of Soft Robotic Fish Driven by Super-Coiled Polymers
LMI Control Design With Input Covariance Constraint for a Tensegrity Simplex Structure
Development of a Compliant Gripper Driven by 3 DOF Soft Robot
Fluid-Driven Soft Actuators for Soft Robots
Kinematics, Dynamics and Control of a Planar 3-DOF Tensegrity Robot Manipulator
Optimal impedance control based on body inertia for a hydraulically driven hexapod robot walking on uneven and extremely soft terrain
Locomotion characteristics of a wheeled vibration-driven robot with an enhanced pantograph-type suspension
Inertia-Driven Controlled Passive Actuation

Top 10 by Citation Count:
Soft robotics: Technologies and systems pushing the boundaries of robot abilities
Soft Robotics: Review of Fluid‐Driven Intrinsically Soft Devices; Manufacturin